In [27]:
from __future__ import print_function, division
#
import sys,os
os.environ['KMP_DUPLICATE_LIB_OK']='True' # uncomment this line if omp error occurs on OSX for python 3
os.environ['OMP_NUM_THREADS']='1' # set number of OpenMP threads to run in parallel
os.environ['MKL_NUM_THREADS']='10' # set number of MKL threads to run in parallel
#
quspin_path = os.path.join(os.getcwd(),"../../")
sys.path.insert(0,quspin_path)
from quspin.basis import spin_basis_1d, spin_basis_general
from quspin.operators import hamiltonian,quantum_operator
from quspin.tools.lanczos import lanczos_full,lanczos_iter,FTLM_static_iteration,LTLM_static_iteration
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

### hisenberg1D

In [ ]:
basis = spin_basis_1d(L,pauli=False)
J_zz = [[1,i,(i+1)%L] for i in range(L)] # OBC
J_xy = [[1/2.0,i,(i+1)%L] for i in range(L)] # OBC
zz = [[1.0,i,(i+1)%L] for i in range(L)]
ops_dict = dict(Jpm=[["+-",J_xy]],Jmp=[["-+",J_xy]],Jzz=[["zz",J_zz]])
H = quantum_operator(ops_dict,basis=basis,dtype=np.float64, check_symm=False)

### heisenberg2D

### use pauli operator

In [28]:
L = 4
Lx, Ly = L, L # linear dimension of spin 1 2d lattice
N_2d = Lx*Ly # number of sites for spin 1
s = np.arange(N_2d) # sites [0,1,2,....]
x = s%Lx # x positions for sites
y = s//Lx # y positions for sites
T_x = (x+1)%Lx + Lx*y # translation along x-direction
T_y = x +Lx*((y+1)%Ly) # translation along y-direction
P_x = x + Lx*(Ly-y-1) # reflection about x-axis
P_y = (Lx-x-1) + Lx*y # reflection about y-axis
Z   = -(s+1) # spin inversion

Jz = 1
J_zz = [[Jz,i,T_x[i]] for i in range(N_2d)]+[[Jz,i,T_y[i]] for i in range(N_2d)]
Jx = 1/2.0
J_xy = [[Jx,i,T_x[i]] for i in range(N_2d)]+[[Jx,i,T_y[i]] for i in range(N_2d)]
ops_dict = dict(Jpm=[["+-",J_xy]],Jmp=[["-+",J_xy]],Jzz=[["zz",J_zz]])

In [54]:
E = []

for nup in range(L**L):
    print(nup)
    basis_2d = spin_basis_general(N_2d, Nup=nup, pauli=0)
    H = quantum_operator(ops_dict,basis=basis_2d, dtype=np.float64, check_symm=False)
    E_= H.eigvalsh({})
    E.append(E_)
E = np.concatenate(E,axis=0)

0
Hermiticity check passed!
Particle conservation check passed!
1
Hermiticity check passed!
Particle conservation check passed!
2
Hermiticity check passed!
Particle conservation check passed!
3
Hermiticity check passed!
Particle conservation check passed!
4
Hermiticity check passed!
Particle conservation check passed!
5
Hermiticity check passed!
Particle conservation check passed!
6
Hermiticity check passed!
Particle conservation check passed!
7
Hermiticity check passed!
Particle conservation check passed!
8
Hermiticity check passed!
Particle conservation check passed!
9
Hermiticity check passed!
Particle conservation check passed!
10
Hermiticity check passed!
Particle conservation check passed!
11
Hermiticity check passed!
Particle conservation check passed!
12
Hermiticity check passed!
Particle conservation check passed!
13
Hermiticity check passed!
Particle conservation check passed!
14
Hermiticity check passed!
Particle conservation check passed!
15
Hermiticity check passed!
Partic

178
Hermiticity check passed!
Particle conservation check passed!
179
Hermiticity check passed!
Particle conservation check passed!
180
Hermiticity check passed!
Particle conservation check passed!
181
Hermiticity check passed!
Particle conservation check passed!
182
Hermiticity check passed!
Particle conservation check passed!
183
Hermiticity check passed!
Particle conservation check passed!
184
Hermiticity check passed!
Particle conservation check passed!
185
Hermiticity check passed!
Particle conservation check passed!
186
Hermiticity check passed!
Particle conservation check passed!
187
Hermiticity check passed!
Particle conservation check passed!
188
Hermiticity check passed!
Particle conservation check passed!
189
Hermiticity check passed!
Particle conservation check passed!
190
Hermiticity check passed!
Particle conservation check passed!
191
Hermiticity check passed!
Particle conservation check passed!
192
Hermiticity check passed!
Particle conservation check passed!
193
Hermit

### expectation value for energy at T = 1

In [55]:
T = 1
beta = 1/T
np.sum(E * np.exp(-beta*E))/np.sum(np.exp(-beta*E)) / (L**2)

-0.4182908067960382

### use approx method (detail [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.eigsh.html) )

In [65]:
basis_2d = spin_basis_general(N_2d, pauli=0)
H = quantum_operator(ops_dict,basis=basis_2d, dtype=np.float64, check_symm=False)
E_eigsh, _ = H.eigsh({}, k = 10)

Hermiticity check passed!


In [66]:
E_eigsh

array([-11.22848321, -10.64988487, -10.64988487, -10.64988487,
        -9.51768798,  -9.51768798,  -9.51768798,  -9.51768798,
        -9.51768798,  -8.88644241])

In [67]:
np.sort(E)[:10]

array([-11.22848321, -10.64988487, -10.64988487, -10.64988487,
        -9.51768798,  -9.51768798,  -9.51768798,  -9.51768798,
        -9.51768798,  -8.88644241])